# Module 11 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [3]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [5]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# Extract all rows of data
table = soup.find('table', class_='table')
rows = table.find_all('tr')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [8]:
# Create an empty list
table_data = []

# Loop through the scraped data to create a list of rows
for row in rows:
    # extract columns
    cols = row.find_all('td')
    # extract and make a list
    col_values = [col.get_text(strip=True) for col in cols]
    # append
    if col_values:
        table_data.append(col_values)

In [17]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
column_headers = [header.get_text(strip=True) for header in table.find_all('th')]
df = pd.DataFrame(table_data, columns=column_headers)

In [ ]:
# show df
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# examine data type
print(df.info())

In [20]:
# change dt for analysis
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date']) 
df['sol'] = df['sol'].astype(int)  
df['ls'] = df['ls'].astype(int)  
df['month'] = df['month'].astype(int) 
df['min_temp'] = df['min_temp'].astype(float)  
df['pressure'] = df['pressure'].astype(float)  

In [ ]:
# Confirm type changes were successful by examining data types again
print(df.info())

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
months = df['month'].nunique()
print(f"There are {months} months on Mars.")

In [ ]:
# 2. How many sols (Martian days) worth of data are there?
sols = df['sol'].count()
print(f"There are {sols} sols of data.")

In [ ]:
# 3. What is the average minimum temperature by month?
avg_min_temp_month = df.groupby('month')['min_temp'].mean()
print(avg_min_temp_month)

In [ ]:
# Plot the average minimum temperature by month
avg_min_temp_month.plot(kind='bar', 
                           title='Average Minimum Temperature by Month', 
                           ylabel='Temperature (°C)', 
                           xlabel='Month', 
                           color='blue', 
                           ylim=(avg_min_temp_month.min() - 5, 0))  # Set the y-axis limits
plt.gca().invert_yaxis()  # Invert the y-axis to make bars appear from the bottom
plt.show()


In [ ]:
# Identify the coldest and hottest months in Curiosity's location by sorting the previous graph

sort_avg_min_temp = avg_min_temp_month.sort_values()

coldest = sort_avg_min_temp.idxmin()
hottest = sort_avg_min_temp.idxmax()

print(f"The coldest month is Month {coldest} with an average minimum temperature of {sort_avg_min_temp.min():.2f}°C.")
print(f"The hottest month is Month {hottest} with an average minimum temperature of {sort_avg_min_temp.max():.2f}°C.")

In [ ]:
# 4. What is the average pressure by month?
avg_pressure_month = df.groupby('month')['pressure'].mean()
print(avg_pressure_month)

In [ ]:
# Plot the average pressure by month
avg_pressure_month.plot(kind='bar', title='Average Atmospheric Pressure by Month', ylabel='Pressure (Pa)', xlabel='Month')
plt.show()

In [ ]:
# Identify the lowest and highest pressure months in Curiosity's location by sorting the previous graph
sort_avg_pressure_month = avg_pressure_month.sort_values()

# lowest and highest months of pressure
lowest = sort_avg_pressure_month.idxmin()
highest = sort_avg_pressure_month.idxmax()

# extract pressure
lowest_pressure_value = sort_avg_pressure_month[lowest]
highest_pressure_value = sort_avg_pressure_month[highest]

# results
print(f"The month with the lowest atmospheric pressure is Month {lowest} with an average pressure of {lowest_pressure_value:.2f} Pa.")
print(f"The month with the highest atmospheric pressure is Month {highest} with an average pressure of {highest_pressure_value:.2f} Pa.")

In [ ]:
# 5. How many terrestrial (Earth) days are there in a Martian year?
# Visually estimate the result by plotting the daily minimum temperature of each observation in the data set.
df_sorted = df.sort_values('terrestrial_date')
plt.plot(df_sorted['terrestrial_date'], df_sorted['min_temp'])
plt.title('Daily Minimum Temperature Over Time')
plt.xlabel('Terrestrial Date')
plt.ylabel('Minimum Temperature (°C)')
plt.show()

#### Minimum Temperature

Mars exhibits extreme temperatures, vividly portraying its harsh environment. The average minimum temperatures illustrate the struggle of the planet’s thin atmosphere to retain heat, particularly during the frigid months. These colder periods underscore the impact of Mars’ lack of atmospheric density, which fails to protect the surface from rapid cooling after sunset. This trend, beyond the numbers, emphasizes the formidable challenge that the Martian climate poses to any potential exploration or settlement. Equipment and habitats would need to be exceptionally resilient to endure the cold and the rapid fluctuations that define the planet’s weather patterns. The data prompts a deeper examination of how these temperature shifts may influence surface activity, such as dust storms or the behavior of carbon dioxide frost.

#### Atmospheric Pressure

The variations in atmospheric pressure across Martian months offer insights into the delicate balance of the planet's environment. Periods of higher pressure suggest moments when atmospheric gases are more concentrated, likely influenced by seasonal or surface dynamics. Lower pressure periods, in contrast, emphasize the tenuous nature of Mars' atmosphere, revealing the challenges of maintaining gas retention under such conditions. Observing these patterns deepens our understanding of the planet's capacity to sustain even minimal atmospheric stability and how external factors shape these cycles.

#### Year Length

Recurring patterns in daily minimum temperatures reveal the length of a Martian year, offering a clear view of the planet's extended orbital rhythm. Peaks and troughs in these temperature cycles trace a year nearly double the length of Earth's. The prolonged timeline influences every aspect of Martian weather, from seasonal shifts to the duration of sol-based climate patterns. Understanding the extended cycle lays a foundation for planning exploratory missions and assessing the practicality of sustaining long-term human or robotic operations on the planet.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [43]:
# Write the data to a CSV
df.to_csv('mars_weather.csv', index=False)

In [44]:
browser.quit()